In [1]:
import pandas as pd
import pathlib
import psutil
import pickle
import numpy as np
import os
import gc
import matplotlib.pyplot as plt
import ta
    

from tqdm.auto import tqdm
from typing import List
from datetime import datetime 
from numba import njit, prange
from itertools import combinations

from ml_utils import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from catboost.utils import get_gpu_device_count

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
MD_PATH = pathlib.Path('/home/jupyter/datasphere/project/')

In [5]:
bbos = pd.read_parquet(MD_PATH / 'bbo_train.parquet')
trades = pd.read_parquet(MD_PATH / 'trades_train.parquet')
lobs = pd.read_parquet(MD_PATH / 'book_train.parquet')

target = pd.read_parquet(MD_PATH / 'target_train.parquet')

In [6]:
bbos.head()

,bid_price,bid_amount,ask_price,ask_amount,midprice
local_ts,,,,,
1699420537200000000,35344.90000,10.60700,35345.00000,2.74100,35344.95000
1699420537400000000,35344.90000,10.61200,35345.00000,2.74100,35344.95000
1699420537500000000,35344.90000,10.57600,35345.00000,2.74900,35344.95000
1699420537600000000,35344.90000,10.62300,35345.00000,2.74900,35344.95000
1699420537700000000,35344.90000,10.60100,35345.00000,2.74900,35344.95000


In [7]:
lobs.head()

,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,bids[1].price,bids[1].amount,asks[2].price,asks[2].amount,bids[2].price,bids[2].amount,asks[3].price,asks[3].amount,bids[3].price,bids[3].amount,asks[4].price,asks[4].amount,bids[4].price,bids[4].amount,asks[5].price,asks[5].amount,bids[5].price,bids[5].amount,asks[6].price,asks[6].amount,bids[6].price,bids[6].amount,asks[7].price,asks[7].amount,bids[7].price,bids[7].amount,asks[8].price,asks[8].amount,bids[8].price,bids[8].amount,asks[9].price,asks[9].amount,bids[9].price,bids[9].amount
local_timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1699420537200000000,35345.00000,2.74100,35344.90000,10.60700,35346.40000,0.03800,35341.80000,0.04500,35346.60000,0.95300,35340.90000,2.89100,35348.80000,0.45200,35337.50000,0.83400,35351.60000,0.11900,35334.70000,0.15800,35355.40000,3.06100,35334.50000,1.74600,35362.90000,2.22200,35333.30000,1.41500,35399.20000,0.21800,35331.80000,0.97400,35404.30000,0.00300,35331.40000,1.88600,35440.80000,0.02200,35330.90000,0.60500
1699420537300000000,35346.20000,0.01200,35344.90000,10.60700,35346.40000,0.03800,35343.10000,0.14700,35346.60000,0.95000,35342.90000,1.98700,35346.80000,0.08400,35342.60000,0.08500,35348.80000,0.45200,35341.80000,0.04200,35351.60000,0.11900,35340.90000,2.89100,35352.50000,0.56900,35339.80000,0.01100,35355.20000,0.02800,35337.50000,0.83100,35355.40000,2.92000,35337.10000,0.94700,35362.90000,2.22200,35336.90000,1.59000
1699420537400000000,35345.00000,2.74100,35344.90000,10.61200,35346.20000,0.00600,35343.10000,0.14600,35346.30000,0.00800,35342.90000,1.98700,35346.40000,0.03800,35342.60000,0.08500,35346.60000,0.95000,35341.80000,0.04200,35346.80000,0.08400,35340.90000,2.89100,35348.80000,0.42500,35339.80000,0.01100,35351.60000,0.11900,35337.50000,0.83100,35352.50000,0.56900,35337.20000,0.35800,35355.20000,0.02800,35337.10000,0.94400
1699420537500000000,35345.00000,2.74900,35344.90000,10.57600,35346.20000,0.00600,35343.10000,0.12900,35346.30000,0.00800,35342.90000,1.98700,35346.40000,0.03800,35342.60000,0.08500,35346.60000,0.95000,35341.80000,0.04200,35346.80000,0.08400,35340.90000,2.89100,35348.80000,0.42500,35339.80000,0.01100,35351.60000,0.11900,35337.50000,0.83100,35352.50000,0.56900,35337.20000,0.35800,35355.20000,0.02800,35337.10000,0.94400
1699420537600000000,35345.00000,2.74900,35344.90000,10.62300,35346.20000,0.00600,35343.10000,0.14600,35346.30000,0.00500,35342.90000,1.98700,35346.40000,0.03800,35342.60000,0.08500,35346.50000,0.02900,35341.80000,0.04200,35346.60000,0.95000,35340.90000,2.89100,35346.80000,0.07900,35339.80000,0.01100,35348.80000,0.42500,35337.50000,0.83100,35351.60000,0.11900,35337.40000,0.00900,35352.50000,0.56900,35337.20000,0.35500


In [8]:
trades.head()

,side,price,amount
local_ts,,,
1699420538721238706,1,35345.00000,0.01000
1699420538721385832,1,35345.00000,0.01000
1699420538723230795,1,35345.00000,0.00200
1699420538723285492,1,35345.00000,0.01400
1699420538723289052,1,35345.00000,0.00400


In [9]:
def calc_my_features(
    bbos: pd.DataFrame,
    lobs: pd.DataFrame,
    trades: pd.DataFrame
                 ) -> pd.DataFrame:

        vimba_up_3_level_df = vimba_up_level(lobs, up_level=3)
        vimba_up_5_level_df = vimba_up_level(lobs, up_level=5)
        vimba_up_10_level_df = vimba_up_level(lobs, up_level=9)

        vimba_at_levels_df = vimba_level(lobs, levels=[0, 1, 2, 3])

        bd_diff_np = book_depth(lobs, size=15, side='ask').values - book_depth(lobs, size=15, side='bid').values
        bd_diff_df = pd.DataFrame({'book_depth_diff_15_btc': bd_diff_np}, index=lobs.index)
        
        mid_price = midprice(lobs)
        vol = volume(lobs)
        liqd_imb = liquidity_imbalance(lobs)
        price_imb = price_imbalance(lobs)
        size_imb = size_imbalance(lobs)
        spread = price_spread(lobs)
        b_spread = bid_spread(lobs)
        a_spread = ask_spread(lobs)
        spread_int = spread_intensity(spread)
        spread_dep = spread_depth_ratio(lobs)
        
        wap_at_0 = wap(lobs, 0)
        wap_at_1 = wap(lobs, 1)
        wap_at_2 = wap(lobs, 2)
        wap_at_3 = wap(lobs, 3)
        #wap_at_4 = wap(lobs, 4)
        wap_at_5 = wap(lobs, 5)
        wap_bal_10 = wap_balance(wap_at_0, wap_at_1, [0, 1])
        wap_bal_20 = wap_balance(wap_at_0, wap_at_2, [0, 2])
        wap_bal_30 = wap_balance(wap_at_0, wap_at_3, [0, 3])
        wap_bal_50 = wap_balance(wap_at_0, wap_at_5, [0, 5])
        wap_bal_21 = wap_balance(wap_at_1, wap_at_2, [1, 2])
        
        market_urg = market_urgency(spread, liqd_imb)
        relative_sp = relative_spread(spread, wap_at_0)
        
        lobs['mid_price'] = mid_price['mid_price']
        mid_price_move = midprice_movement(lobs)
        
        lobs['wap_at_0'] = wap_at_0['wap_at_0']
        lobs['wap_at_1'] = wap_at_1['wap_at_1']
        lobs['wap_at_2'] = wap_at_2['wap_at_2']
        lobs['wap_at_3'] = wap_at_3['wap_at_3']
        
        prices = ["asks[0].price", "bids[0].price", "asks[1].price", "bids[1].price", 
                  "wap_at_0", "wap_at_1", "wap_at_2", "wap_at_3"]
        sizes = ["bids[0].amount", "asks[0].amount", "bids[1].amount", "asks[1].amount"]
        
        triplet_imb_price = calculate_triplet_imbalance_numba(prices, lobs)
        triplet_imb_size = calculate_triplet_imbalance_numba(sizes, lobs)
        
        cols = ["asks[0].price", "bids[0].price", "bids[0].amount", "asks[0].amount"]
        diff_features = differences(lobs, cols, [1, 2, 3, 5, 10])
        
        return pd.concat([
            vimba_up_3_level_df,
            vimba_up_5_level_df,
            vimba_up_10_level_df,
            vimba_at_levels_df,
            bd_diff_df,
            mid_price,
            vol,
            liqd_imb,
            price_imb,
            size_imb,
            spread,
            b_spread,
            a_spread,
            spread_int,
            spread_dep,
            wap_at_0,
            wap_at_1,
            wap_at_2,
            wap_at_3,
            #wap_at_4,
            wap_at_5,
            wap_bal_10,
            wap_bal_20,
            wap_bal_30,
            wap_bal_50,
            wap_bal_21,
            market_urg,
            relative_sp,
            mid_price_move,
            triplet_imb_price,
            triplet_imb_size,
            diff_features
        ], axis=1).asof(bbos.index)

In [10]:
features_df = calc_my_features(bbos=bbos, lobs=lobs, trades=trades)

/usr/local/lib/python3.10/dist-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'comb_indices' of function 'compute_triplet_imbalance'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "ml_utils.py", line 161:
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/usr/local/lib/python3.10/dist-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'comb__indices' of function '__numba_parfor_gufunc_0x7fc2cd1661a0'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types



In [11]:
features_df

,vimba_up_3,vimba_up_5,vimba_up_9,vimba_at_0,vimba_at_1,vimba_at_2,vimba_at_3,book_depth_diff_15_btc,mid_price,volume,liquidity_imbalance,price_imbalance,size_imbalance,price_spread,bid_spread,ask_spread,spread_intensity,spread_depth_ratio,wap_at_0,wap_at_1,wap_at_2,wap_at_3,wap_at_5,wap_balance_01,wap_balance_02,wap_balance_03,wap_balance_05,wap_balance_12,market_urgency,relative_spread,mid_price_movement,asks[0].price_bids[0].price_asks[1].price_imb2,asks[0].price_bids[0].price_bids[1].price_imb2,asks[0].price_bids[0].price_wap_at_0_imb2,asks[0].price_bids[0].price_wap_at_1_imb2,asks[0].price_bids[0].price_wap_at_2_imb2,asks[0].price_bids[0].price_wap_at_3_imb2,asks[0].price_asks[1].price_bids[1].price_imb2,asks[0].price_asks[1].price_wap_at_0_imb2,asks[0].price_asks[1].price_wap_at_1_imb2,asks[0].price_asks[1].price_wap_at_2_imb2,asks[0].price_asks[1].price_wap_at_3_imb2,asks[0].price_bids[1].price_wap_at_0_imb2,asks[0].price_bids[1].price_wap_at_1_imb2,asks[0].price_bids[1].price_wap_at_2_imb2,asks[0].price_bids[1].price_wap_at_3_imb2,asks[0].price_wap_at_0_wap_at_1_imb2,asks[0].price_wap_at_0_wap_at_2_imb2,asks[0].price_wap_at_0_wap_at_3_imb2,asks[0].price_wap_at_1_wap_at_2_imb2,asks[0].price_wap_at_1_wap_at_3_imb2,asks[0].price_wap_at_2_wap_at_3_imb2,bids[0].price_asks[1].price_bids[1].price_imb2,bids[0].price_asks[1].price_wap_at_0_imb2,bids[0].price_asks[1].price_wap_at_1_imb2,bids[0].price_asks[1].price_wap_at_2_imb2,bids[0].price_asks[1].price_wap_at_3_imb2,bids[0].price_bids[1].price_wap_at_0_imb2,bids[0].price_bids[1].price_wap_at_1_imb2,bids[0].price_bids[1].price_wap_at_2_imb2,bids[0].price_bids[1].price_wap_at_3_imb2,bids[0].price_wap_at_0_wap_at_1_imb2,bids[0].price_wap_at_0_wap_at_2_imb2,bids[0].price_wap_at_0_wap_at_3_imb2,bids[0].price_wap_at_1_wap_at_2_imb2,bids[0].price_wap_at_1_wap_at_3_imb2,bids[0].price_wap_at_2_wap_at_3_imb2,asks[1].price_bids[1].price_wap_at_0_imb2,asks[1].price_bids[1].price_wap_at_1_imb2,asks[1].price_bids[1].price_wap_at_2_imb2,asks[1].price_bids[1].price_wap_at_3_imb2,asks[1].price_wap_at_0_wap_at_1_imb2,asks[1].price_wap_at_0_wap_at_2_imb2,asks[1].price_wap_at_0_wap_at_3_imb2,asks[1].price_wap_at_1_wap_at_2_imb2,asks[1].price_wap_at_1_wap_at_3_imb2,asks[1].price_wap_at_2_wap_at_3_imb2,bids[1].price_wap_at_0_wap_at_1_imb2,bids[1].price_wap_at_0_wap_at_2_imb2,bids[1].price_wap_at_0_wap_at_3_imb2,bids[1].price_wap_at_1_wap_at_2_imb2,bids[1].price_wap_at_1_wap_at_3_imb2,bids[1].price_wap_at_2_wap_at_3_imb2,wap_at_0_wap_at_1_wap_at_2_imb2,wap_at_0_wap_at_1_wap_at_3_imb2,wap_at_0_wap_at_2_wap_at_3_imb2,wap_at_1_wap_at_2_wap_at_3_imb2,bids[0].amount_asks[0].amount_bids[1].amount_imb2,bids[0].amount_asks[0].amount_asks[1].amount_imb2,bids[0].amount_bids[1].amount_asks[1].amount_imb2,asks[0].amount_bids[1].amount_asks[1].amount_imb2,asks[0].price_diff_1,asks[0].price_diff_2,asks[0].price_diff_3,asks[0].price_diff_5,asks[0].price_diff_10,bids[0].price_diff_1,bids[0].price_diff_2,bids[0].price_diff_3,bids[0].price_diff_5,bids[0].price_diff_10,bids[0].amount_diff_1,bids[0].amount_diff_2,bids[0].amount_diff_3,bids[0].amount_diff_5,bids[0].amount_diff_10,asks[0].amount_diff_1,asks[0].amount_diff_2,asks[0].amount_diff_3,asks[0].amount_diff_5,asks[0].amount_diff_10
local_ts,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1699420537200000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1699420537400000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [12]:
device = 'GPU' if get_gpu_device_count() > 0 else 'CPU'

Error 100 (no CUDA-capable device is detected) ignored while obtaining device count


In [13]:
device

'CPU'

In [14]:
model_params = {
    'depth': 5,
    'iterations': 1200,   
    'thread_count': 7,
    'learning_rate': 0.01,
    'l2_leaf_reg': 50,

    'task_type': device,
    'eval_metric': 'R2',
    
    'use_best_model': True,
    'silent': True,
}

#### Estimate quality of model 

In [15]:
model = CatBoostRegressor(**model_params)

X_train, X_test, y_train, y_test = train_test_split(features_df, target, test_size=0.2, random_state=42, shuffle=False)

# 

model.fit(X=X_train, y=y_train, eval_set=(X_test, y_test), plot=True)q

score = r2_score(y_test, model.predict(X_test))
print("best_score:", score)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

best_score: 0.03653707731954725


In [16]:
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

In [17]:
importances = model.get_feature_importance(train_pool, type='PredictionValuesChange')
feature_importances = pd.Series(importances, index=features_df.columns).sort_values()

In [18]:
feature_importances 

wap_at_0_wap_at_1_wap_at_2_imb2                      0.00000
wap_balance_01                                       0.00000
bids[1].price_wap_at_1_wap_at_3_imb2                 0.00000
asks[0].price_bids[0].price_wap_at_1_imb2            0.00000
wap_balance_03                                       0.00000
relative_spread                                      0.00000
price_imbalance                                      0.00000
mid_price_movement                                   0.00000
wap_balance_05                                       0.00169
bids[0].amount_bids[1].amount_asks[1].amount_imb2    0.00174
vimba_at_3                                           0.00414
asks[1].price_bids[1].price_wap_at_0_imb2            0.00555
asks[0].price_wap_at_1_wap_at_2_imb2                 0.00561
bids[1].price_wap_at_1_wap_at_2_imb2                 0.00636
wap_balance_02                                       0.00661
spread_depth_ratio                                   0.00672
bids[1].price_wap_at_2_w

In [31]:
list(feature_importances.index[-50:])

['asks[0].price_bids[1].price_wap_at_3_imb2',
 'asks[0].price_bids[1].price_wap_at_0_imb2',
 'wap_at_0_wap_at_1_wap_at_3_imb2',
 'asks[0].price_asks[1].price_bids[1].price_imb2',
 'bids[0].amount_diff_10',
 'bids[0].price_diff_2',
 'asks[0].price_asks[1].price_wap_at_0_imb2',
 'bids[1].price_wap_at_0_wap_at_1_imb2',
 'bids[0].amount_diff_2',
 'asks[1].price_wap_at_0_wap_at_3_imb2',
 'asks[0].amount_diff_10',
 'bids[0].price_wap_at_0_wap_at_2_imb2',
 'bids[0].amount_diff_3',
 'ask_spread',
 'asks[0].amount_diff_5',
 'asks[0].amount_diff_2',
 'asks[0].amount_diff_3',
 'asks[0].amount_diff_1',
 'bids[0].amount_diff_5',
 'spread_intensity',
 'bids[0].price_asks[1].price_bids[1].price_imb2',
 'bids[0].amount_diff_1',
 'bids[0].price_diff_1',
 'asks[0].price_diff_3',
 'asks[0].price_bids[0].price_bids[1].price_imb2',
 'bids[0].price_wap_at_0_wap_at_3_imb2',
 'asks[0].price_diff_5',
 'bid_spread',
 'asks[0].price_wap_at_0_wap_at_2_imb2',
 'price_spread',
 'asks[0].price_wap_at_0_wap_at_3_imb2

In [28]:
top50_features = features_df[list(feature_importances.index[-50:])]

In [29]:
X_train, X_test, y_train, y_test = train_test_split(top50_features, target, test_size=0.2, random_state=42, shuffle=False)

# 

model.fit(X=X_train, y=y_train, eval_set=(X_test, y_test), plot=True)

score = r2_score(y_test, model.predict(X_test))
print("best_score:", score)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

best_score: 0.0365150851187348
